# Text mining & Search Project

### Università degli Studi di Milano-Bicocca  2020/2021

**Luzzi Federico** (matricola) **Peracchi Marco** 800578

# Text Exploration

In questa fase procediamo all'esplorazione dei dati a disposizione.

### Librerie

In [3]:
# Librerie base
import nltk
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import sklearn
from wordcloud import WordCloud

In [4]:
# Librerie per la text tokenization
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  WordPunctTokenizer
from nltk.tokenize import  BlanklineTokenizer

In [5]:
# Librerie per stemming e lemmatization
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [6]:
# Librerie per text representation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
# Download dei contenuti necessari
nltk.download("stopwords")
stop_words = nltk.corpus.stopwords.words("english")
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Marco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Dataset

Il dataset a disposizione presenta diverse colonne:
- **count** rappresenta il numero di utenti che hanno espresso una valutazione sul tweet
- **hate_speech** è il numero di utenti che hanno ritenuto il tweet come espressioni violente 
- **offensive_language** sono gli utenti che hanno segnalato come tweet offensivo
- **neither** sono gli utenti che hanno segnalato il tweet come neutrale
- **class** è la label assegnata, 0 come hate speech, 1 per offensive language e 2 come neutrale
- **tweet** rappresenta il testo del tweet

In [8]:
df = pd.read_csv("data/labeled_data.csv", sep = ',').drop("Unnamed: 0", axis=1)
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [10]:
# Formato delle colonne
df.dtypes

count                  int64
hate_speech            int64
offensive_language     int64
neither                int64
class                  int64
tweet                 object
dtype: object

In [11]:
# Statistiche descrittive del dataset
df.describe()

,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [12]:
# Distribuzione delle label
df["class"].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

Dalla distribuzione delle label è chiaramente visibile un problema di sbilanciamento delle classi.

In [16]:
# Esempio di un tweet segnalato come neutrale
df["tweet"].loc[6171]

'@infidelpamelaLC I\'m going to blame the black man, since they always blame "whitey" I\'m an equal opportunity hater.'

In [17]:
df.loc[6171]

count                                                                 9
hate_speech                                                           7
offensive_language                                                    1
neither                                                               1
class                                                                 0
tweet                 @infidelpamelaLC I'm going to blame the black ...
Name: 6171, dtype: object

### Interesting rows

- 1432
- 24
- 3129
- 530
- 19999
- 998
- 1605
- 4567
- 555